# Homework (deadline 18.11.2022 13:44:59)
Write solutions for the homework exercises in this notebook. Once the work is done download the notebook file (`File > Download .ipynb`) rename it properly so it follows a template `HW5_<SURNAME>_<NAME>.ipynb` and send the file to me. Please also attach all files you will be asked to produce in this homework. My email address is as follows: 

* <m.biesaga@uw.edu.pl>

Remember that you can contact me via email if you have any problems. Moreover, you can also visit me in the ISS on the fourth floor (room 415). Usually, I am there from 11ish but please let me know in advance if you are coming because I might be busy. 


## Task 1 (5 points)

Read about the `pageviews` method (`prop=pageviews`) in the `query endpoint` ([docpage](https://en.wikipedia.org/w/api.php?action=help&modules=query%2Bpageviews)). Use this method to extract page views data for the pages from the previous exercise (if you want you can sample 10 new pages with the `list=random` method) for the last 60 days. The results will be broken down by single days, so you have to aggregate the results (sum) so they give the total page views count for the entire period of 60 days. Remember that to select pages by page ids you pass `pageids=<id 1>|<id 2>|...|<id n>`. We did a very similar thing when we extracted article content through the `cirrusdoc` method in the Wikipedia API in the previous part of this notebook. Your final output should be a `dict` object that maps page ids to pageviews (total number of pageviews over 60 days). It should look something like this:

```python
results = {
    # page_id: pageviews
    153253: 10204,
    423423: 101,
    11012:  12,
    42435:  546,
    # and so on
}
```

If you want you can sample 10 pages yourself. Otherwise, you may use the following list of page ids that we prepared for you. Sampling pages yourself will give you extra credit (but it is possible to get maximum points without it as well).

In [129]:
## Import module requests
import requests

## Some page ids
page_ids = [
    19969580,
    39982842,
    25699035,
    52642931,
    53055349,
    24133565,
    1164662,
    40656459,
    12533026,
    47110862
]

## API URL
BASE_URL = 'https://en.wikipedia.org/w/api.php'

In [130]:
params = {
    'action': 'query',
    'prop': 'pageviews',
    'pageids': '|'.join(str(pid) for pid in page_ids),
    'pvidays': 60,
    'format': 'json'
}
response = requests.get(BASE_URL, params=params)
response


data = response.json()['query']['pages']
## My friends solution
PVS = { v['title']: sum(filter(None, v.get('pageviews', {}).values())) for v in data.values() }
## My solutions
PVM = { v['title'] : sum([ item for item in v['pageviews'].values() if item is not None ]) for v in data.values() }
PVS == PVM

True

Taks 2 (5 + 2 points)
In this task, you can score either 5 points or 7. The only difference is in the pages you will download. To score 5 points you just need to download the content of 20 random pages from Wikipedia (please review the [N1](https://github.com/MikoBie/ids/blob/main/notebooks/N1.ipynb) in which we downloaded the content of 10 random pages). To have the chance to score 7 points you need to download the content of 10 pages that have in the title `Olivia` and 10 pages that have `Noah` (those are the most popular names in the UK in 2021). 

**Hint for 7 points**: you might find this [`pssearch`](https://en.wikipedia.org/w/api.php?action=help&modules=query%2Bprefixsearch) method interesting.

When you have the content of these 20 articles for each one of them compute a distribution of the following possessive pronouns: `her`, `his`, and `their`. In other words, you should end up with the list looking more or less like this one:

```python
[{'his': 79, 'her': 212, 'their': 14},
 {'his': 36, 'her': 147, 'their': 20},
 {'his': 17, 'her': 80, 'their': 6},
 {'his': 8, 'her': 80, 'their': 9},
 {'his': 14, 'her': 66, 'their': 2},
 {'his': 12, 'her': 188, 'their': 16},
 {'his': 3, 'her': 156, 'their': 13},
 {'his': 33, 'her': 126, 'their': 33},
 {'his': 10, 'her': 113, 'their': 8},
 {'his': 21, 'her': 4, 'their': 33}]
```
**Hint**: Remember that sometimes in articles pronouns starts with the capital letter (ignore the cases like `hers` and `theirs`). Moreover, review the [notebook number 5](https://github.com/MikoBie/ppss/blob/main/notebooks/N5.ipynb) about the lists from May 19th, 2022.

In [ ]:
## Import module requests
import requests
BASE_URL = 'https://en.wikipedia.org/w/api.php'

In [ ]:
## YOUR CODE

In [ ]:
import requests as rq
URL = 'https://en.wikipedia.org/w/api.php'
payload = { 'action' : 'query',
            'list' : 'prefixsearch',
            'pssearch' : 'Anne',
            'format' : 'json',
            'pslimit' : 11
}

            

In [ ]:
response = rq.get(URL, payload)
data = response.json()

In [ ]:
ids = [ item['pageid'] for item in data['query']['prefixsearch'][1:] ]
names = [ item['title'] for item in data['query']['prefixsearch'][1:] ]
ids_str = '|'.join( str(item) for item in ids )

In [ ]:
payload = { 'action' : 'query',
            'prop' : 'cirrusdoc',
            'pageids' : ids_str,
            'format' : 'json'
}

In [ ]:
response = rq.get(URL, payload)
data = response.json()

In [ ]:
articles = [ p['cirrusdoc'][0]['source']['text'] for p in data['query']['pages'].values() ]
named_articles = zip(names, articles)

In [ ]:
men_pronouns = []
for k, item in named_articles:
    pronouns = { 'his' : 0,
             'her' : 0,
             'their' : 0
             }
    l = item.lower().split()
    pronouns['his'] = l.count('his')    
    pronouns['her'] = l.count('her')    
    pronouns['their'] = l.count('their')    
    men_pronouns.append({k : pronouns })

In [ ]:
women_pronouns

In [ ]:
men_pronouns